In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

In [2]:
# Define the LEMCell
class LEMCell(nn.Module):
    def __init__(self, ninp, nhid, dt):
        super(LEMCell, self).__init__()
        self.ninp = ninp
        self.nhid = nhid
        self.dt = dt
        self.inp2hid = nn.Linear(ninp, 4 * nhid)
        self.hid2hid = nn.Linear(nhid, 3 * nhid)
        self.transform_z = nn.Linear(nhid, nhid)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / np.sqrt(self.nhid)
        for w in self.parameters():
            w.data.uniform_(-std, std)

    def forward(self, x, y, z):
        transformed_inp = self.inp2hid(x)
        transformed_hid = self.hid2hid(y)
        i_dt1, i_dt2, i_z, i_y = transformed_inp.chunk(4, 1)
        h_dt1, h_dt2, h_y = transformed_hid.chunk(3, 1)

        ms_dt_bar = self.dt * torch.sigmoid(i_dt1 + h_dt1)
        ms_dt = self.dt * torch.sigmoid(i_dt2 + h_dt2)

        z = (1. - ms_dt) * z + ms_dt * torch.tanh(i_y + h_y)
        y = (1. - ms_dt_bar) * y + ms_dt_bar * torch.tanh(self.transform_z(z) + i_z)

        return y, z

# Define the LEM model
class LEM(nn.Module):
    def __init__(self, ninp, nhid, nout, dt=1.):
        super(LEM, self).__init__()
        self.nhid = nhid
        self.cell = LEMCell(ninp, nhid, dt)
        self.classifier = nn.Linear(nhid, nout)
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'classifier' in name and 'weight' in name:
                nn.init.kaiming_normal_(param.data)

    def forward(self, input):
        y = input.data.new(input.size(1), self.nhid).zero_()
        z = input.data.new(input.size(1), self.nhid).zero_()
        for x in input:
            y, z = self.cell(x, y, z)
        out = self.classifier(y)
        return out


### PINN data importing

In [3]:
# importing data

# Load the .mat file
mat_data = scipy.io.loadmat('burg.mat')

# Access the variables stored in the .mat file
# The variable names in the .mat file become keys in the loaded dictionary
x = mat_data['x']
t = mat_data['t']
u = mat_data['u1']



### Exact Solution data importing

In [4]:
# importing data

import torch
import torch.nn as nn
import numpy as np
import time
import scipy.io

# Load the .mat file
mat_data = scipy.io.loadmat('burgers_shock.mat')

# Access the variables stored in the .mat file
# The variable names in the .mat file become keys in the loaded dictionary
x = mat_data['x']
t = mat_data['t']
u_1 = mat_data['usol']


In [5]:
# Set random seed for reproducibility
#torch.manual_seed(42)

# Toy problem data
input_size = 256
hidden_size = 32
output_size = 256
sequence_length = 79
batch_size = 1
num_epochs = 20000

# Set random seed for reproducibility
#torch.manual_seed(42)
u[:, 0:100].shape

(256, 100)

In [6]:
input_data = u[:,0:79]
target_data = u[:,1:80]

test_data = u[:,79]
#test_target = u[:,80:100]

print("test data shape", test_data.shape)
#print("test target shape", test_target.shape)

print("input data shape",input_data.shape)
print("Target data shape",target_data.shape)

# Convert data to tensors
input_tensor = torch.tensor(input_data.T).view(batch_size, sequence_length, input_size).float()
target_tensor = torch.tensor(target_data.T).view(batch_size, sequence_length, output_size).float()

print("input tensor shape",input_tensor.shape)
print("Target tensor shape",target_tensor.shape)

test data shape (256,)
input data shape (256, 79)
Target data shape (256, 79)
input tensor shape torch.Size([1, 79, 256])
Target tensor shape torch.Size([1, 79, 256])


In [7]:
# Convert test data to tensors
test_tensor = torch.tensor(test_data.T).view(batch_size, 1, input_size).float()
#test_target_tensor = torch.tensor(test_target.T).view(batch_size, 20, output_size).float()
target_tensor = torch.squeeze(target_tensor)

In [8]:
# Create LEM instance
lem = LEM(input_size, hidden_size, output_size, dt=0.5)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(lem.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    output = lem(input_tensor)
    loss = criterion(output, target_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.16f}')

Epoch: 10/20000, Loss: 0.3944522142410278
Epoch: 20/20000, Loss: 0.3297334909439087
Epoch: 30/20000, Loss: 0.2621820569038391
Epoch: 40/20000, Loss: 0.1952804625034332
Epoch: 50/20000, Loss: 0.1376726478338242
Epoch: 60/20000, Loss: 0.0936740189790726
Epoch: 70/20000, Loss: 0.0634541064500809
Epoch: 80/20000, Loss: 0.0443493351340294
Epoch: 90/20000, Loss: 0.0329024419188499
Epoch: 100/20000, Loss: 0.0261302292346954
Epoch: 110/20000, Loss: 0.0220247283577919
Epoch: 120/20000, Loss: 0.0194247160106897
Epoch: 130/20000, Loss: 0.0176618527621031
Epoch: 140/20000, Loss: 0.0163477454334497
Epoch: 150/20000, Loss: 0.0152578009292483
Epoch: 160/20000, Loss: 0.0142612718045712
Epoch: 170/20000, Loss: 0.0133048836141825
Epoch: 180/20000, Loss: 0.0123672429472208
Epoch: 190/20000, Loss: 0.0114403245970607
Epoch: 200/20000, Loss: 0.0105259018018842
Epoch: 210/20000, Loss: 0.0096346139907837
Epoch: 220/20000, Loss: 0.0087813707068563
Epoch: 230/20000, Loss: 0.0079760337248445
Epoch: 240/20000, Lo

Epoch: 2070/20000, Loss: 0.0001105943520088
Epoch: 2080/20000, Loss: 0.0001094810577342
Epoch: 2090/20000, Loss: 0.0001085990734282
Epoch: 2100/20000, Loss: 0.0001079358044080
Epoch: 2110/20000, Loss: 0.0001072376908269
Epoch: 2120/20000, Loss: 0.0001065632241080
Epoch: 2130/20000, Loss: 0.0001059204587364
Epoch: 2140/20000, Loss: 0.0001052902152878
Epoch: 2150/20000, Loss: 0.0001046773686539
Epoch: 2160/20000, Loss: 0.0001040794377332
Epoch: 2170/20000, Loss: 0.0001034958040691
Epoch: 2180/20000, Loss: 0.0001029260020005
Epoch: 2190/20000, Loss: 0.0001023695658660
Epoch: 2200/20000, Loss: 0.0001018259863486
Epoch: 2210/20000, Loss: 0.0001012948778225
Epoch: 2220/20000, Loss: 0.0001007759419736
Epoch: 2230/20000, Loss: 0.0001002781573334
Epoch: 2240/20000, Loss: 0.0001014671579469
Epoch: 2250/20000, Loss: 0.0001435351441614
Epoch: 2260/20000, Loss: 0.0001020496347337
Epoch: 2270/20000, Loss: 0.0001003511060844
Epoch: 2280/20000, Loss: 0.0000989845211734
Epoch: 2290/20000, Loss: 0.00009

Epoch: 4090/20000, Loss: 0.0001444946828997
Epoch: 4100/20000, Loss: 0.0000645342806820
Epoch: 4110/20000, Loss: 0.0000586377063883
Epoch: 4120/20000, Loss: 0.0000513248924108
Epoch: 4130/20000, Loss: 0.0000515207721037
Epoch: 4140/20000, Loss: 0.0000506965843670
Epoch: 4150/20000, Loss: 0.0000501514841744
Epoch: 4160/20000, Loss: 0.0000498943809362
Epoch: 4170/20000, Loss: 0.0000495865206176
Epoch: 4180/20000, Loss: 0.0000493275656481
Epoch: 4190/20000, Loss: 0.0000490559614263
Epoch: 4200/20000, Loss: 0.0000487896977575
Epoch: 4210/20000, Loss: 0.0000485252094222
Epoch: 4220/20000, Loss: 0.0000482600444229
Epoch: 4230/20000, Loss: 0.0000479950831505
Epoch: 4240/20000, Loss: 0.0000477299436170
Epoch: 4250/20000, Loss: 0.0000474646440125
Epoch: 4260/20000, Loss: 0.0000471991261293
Epoch: 4270/20000, Loss: 0.0000469333936053
Epoch: 4280/20000, Loss: 0.0000466674355266
Epoch: 4290/20000, Loss: 0.0000464012664452
Epoch: 4300/20000, Loss: 0.0000461348899989
Epoch: 4310/20000, Loss: 0.00004

Epoch: 6000/20000, Loss: 0.0000160130512086
Epoch: 6010/20000, Loss: 0.0000132809691422
Epoch: 6020/20000, Loss: 0.0000129009058583
Epoch: 6030/20000, Loss: 0.0000128622223201
Epoch: 6040/20000, Loss: 0.0000126836721392
Epoch: 6050/20000, Loss: 0.0000125516844491
Epoch: 6060/20000, Loss: 0.0000124623147713
Epoch: 6070/20000, Loss: 0.0000123736390378
Epoch: 6080/20000, Loss: 0.0000122889896375
Epoch: 6090/20000, Loss: 0.0000122060473586
Epoch: 6100/20000, Loss: 0.0000121249213407
Epoch: 6110/20000, Loss: 0.0000120451695693
Epoch: 6120/20000, Loss: 0.0000119664737213
Epoch: 6130/20000, Loss: 0.0000118888792713
Epoch: 6140/20000, Loss: 0.0000118122934509
Epoch: 6150/20000, Loss: 0.0000117367235362
Epoch: 6160/20000, Loss: 0.0000116621213238
Epoch: 6170/20000, Loss: 0.0000115885186460
Epoch: 6180/20000, Loss: 0.0000115159054985
Epoch: 6190/20000, Loss: 0.0000114478771138
Epoch: 6200/20000, Loss: 0.0000118788466352
Epoch: 6210/20000, Loss: 0.0000922292383621
Epoch: 6220/20000, Loss: 0.00001

Epoch: 7910/20000, Loss: 0.0000067261289587
Epoch: 7920/20000, Loss: 0.0000068222857408
Epoch: 7930/20000, Loss: 0.0000219775083679
Epoch: 7940/20000, Loss: 0.0000249832792178
Epoch: 7950/20000, Loss: 0.0000088708329713
Epoch: 7960/20000, Loss: 0.0000074179747571
Epoch: 7970/20000, Loss: 0.0000068929466579
Epoch: 7980/20000, Loss: 0.0000067717805905
Epoch: 7990/20000, Loss: 0.0000067774244599
Epoch: 8000/20000, Loss: 0.0000067213918555
Epoch: 8010/20000, Loss: 0.0000066495681494
Epoch: 8020/20000, Loss: 0.0000066288830567
Epoch: 8030/20000, Loss: 0.0000066151733336
Epoch: 8040/20000, Loss: 0.0000065997760430
Epoch: 8050/20000, Loss: 0.0000065873832682
Epoch: 8060/20000, Loss: 0.0000065750718932
Epoch: 8070/20000, Loss: 0.0000065660597102
Epoch: 8080/20000, Loss: 0.0000067326132012
Epoch: 8090/20000, Loss: 0.0000202556311706
Epoch: 8100/20000, Loss: 0.0000105168155642
Epoch: 8110/20000, Loss: 0.0000076358528531
Epoch: 8120/20000, Loss: 0.0000066379052441
Epoch: 8130/20000, Loss: 0.00000

...

Epoch: 11710/20000, Loss: 0.0000030522123780
Epoch: 11720/20000, Loss: 0.0000086834388640
Epoch: 11730/20000, Loss: 0.0000159576757142
Epoch: 11740/20000, Loss: 0.0000080708932728
Epoch: 11750/20000, Loss: 0.0000040216627895
Epoch: 11760/20000, Loss: 0.0000033479193462
Epoch: 11770/20000, Loss: 0.0000028656966151
Epoch: 11780/20000, Loss: 0.0000027646119634
Epoch: 11790/20000, Loss: 0.0000027789649266
Epoch: 11800/20000, Loss: 0.0000027549581318
Epoch: 11810/20000, Loss: 0.0000028243541692
Epoch: 11820/20000, Loss: 0.0000051459678616
Epoch: 11830/20000, Loss: 0.0000124468333524
Epoch: 11840/20000, Loss: 0.0000050859835028
Epoch: 11850/20000, Loss: 0.0000048544693527
Epoch: 11860/20000, Loss: 0.0000031375107028
Epoch: 11870/20000, Loss: 0.0000029266748243
Epoch: 11880/20000, Loss: 0.0000026969498776
Epoch: 11890/20000, Loss: 0.0000026899119803
Epoch: 11900/20000, Loss: 0.0000027040441637
Epoch: 11910/20000, Loss: 0.0000031380147902
Epoch: 11920/20000, Loss: 0.0000168013248185
Epoch: 119

Epoch: 13580/20000, Loss: 0.0000021742966965
Epoch: 13590/20000, Loss: 0.0000021190539883
Epoch: 13600/20000, Loss: 0.0000018528093051
Epoch: 13610/20000, Loss: 0.0000018755735027
Epoch: 13620/20000, Loss: 0.0000021867772375
Epoch: 13630/20000, Loss: 0.0000068721524258
Epoch: 13640/20000, Loss: 0.0000066227439675
Epoch: 13650/20000, Loss: 0.0000032713080600
Epoch: 13660/20000, Loss: 0.0000021712201033
Epoch: 13670/20000, Loss: 0.0000019385768155
Epoch: 13680/20000, Loss: 0.0000018366406493
Epoch: 13690/20000, Loss: 0.0000021822122562
Epoch: 13700/20000, Loss: 0.0000076987062130
Epoch: 13710/20000, Loss: 0.0000036288420233
Epoch: 13720/20000, Loss: 0.0000055760733630
Epoch: 13730/20000, Loss: 0.0000067507662607
Epoch: 13740/20000, Loss: 0.0000031478834899
Epoch: 13750/20000, Loss: 0.0000020150205273
Epoch: 13760/20000, Loss: 0.0000017619672690
Epoch: 13770/20000, Loss: 0.0000017757744217
Epoch: 13780/20000, Loss: 0.0000020079994556
Epoch: 13790/20000, Loss: 0.0000043863383326
Epoch: 138

Epoch: 15500/20000, Loss: 0.0000021616774575
Epoch: 15510/20000, Loss: 0.0000017866965436
Epoch: 15520/20000, Loss: 0.0000029886950870
Epoch: 15530/20000, Loss: 0.0000066526476985
Epoch: 15540/20000, Loss: 0.0000044933217396
Epoch: 15550/20000, Loss: 0.0000014872787233
Epoch: 15560/20000, Loss: 0.0000016217352368
Epoch: 15570/20000, Loss: 0.0000014124183281
Epoch: 15580/20000, Loss: 0.0000013050998859
Epoch: 15590/20000, Loss: 0.0000015776779492
Epoch: 15600/20000, Loss: 0.0000094586039268
Epoch: 15610/20000, Loss: 0.0000059379776758
Epoch: 15620/20000, Loss: 0.0000057660422499
Epoch: 15630/20000, Loss: 0.0000019464951038
Epoch: 15640/20000, Loss: 0.0000013979439473
Epoch: 15650/20000, Loss: 0.0000014462442550
Epoch: 15660/20000, Loss: 0.0000012443223341
Epoch: 15670/20000, Loss: 0.0000011954091406
Epoch: 15680/20000, Loss: 0.0000011895031093
Epoch: 15690/20000, Loss: 0.0000012003669099
Epoch: 15700/20000, Loss: 0.0000018136794324
Epoch: 15710/20000, Loss: 0.0000085642113845
Epoch: 157

Epoch: 17390/20000, Loss: 0.0000008668019404
Epoch: 17400/20000, Loss: 0.0000009338486961
Epoch: 17410/20000, Loss: 0.0000015671963638
Epoch: 17420/20000, Loss: 0.0000070839269029
Epoch: 17430/20000, Loss: 0.0000011022301578
Epoch: 17440/20000, Loss: 0.0000010531072121
Epoch: 17450/20000, Loss: 0.0000010917144664
Epoch: 17460/20000, Loss: 0.0000008390992434
Epoch: 17470/20000, Loss: 0.0000007547563428
Epoch: 17480/20000, Loss: 0.0000010085776694
Epoch: 17490/20000, Loss: 0.0000064610530899
Epoch: 17500/20000, Loss: 0.0000160457457241
Epoch: 17510/20000, Loss: 0.0000042526248762
Epoch: 17520/20000, Loss: 0.0000021544656192
Epoch: 17530/20000, Loss: 0.0000010034834759
Epoch: 17540/20000, Loss: 0.0000007928983905
Epoch: 17550/20000, Loss: 0.0000007130230415
Epoch: 17560/20000, Loss: 0.0000006705361102
Epoch: 17570/20000, Loss: 0.0000006637576462
Epoch: 17580/20000, Loss: 0.0000006582152423
Epoch: 17590/20000, Loss: 0.0000006923592082
Epoch: 17600/20000, Loss: 0.0000034429569951
Epoch: 176

Epoch: 19260/20000, Loss: 0.0000044193739086
Epoch: 19270/20000, Loss: 0.0000018917644411
Epoch: 19280/20000, Loss: 0.0000009216243484
Epoch: 19290/20000, Loss: 0.0000006867124398
Epoch: 19300/20000, Loss: 0.0000007279468264
Epoch: 19310/20000, Loss: 0.0000028446220313
Epoch: 19320/20000, Loss: 0.0000077261101978
Epoch: 19330/20000, Loss: 0.0000034741142372
Epoch: 19340/20000, Loss: 0.0000015509666582
Epoch: 19350/20000, Loss: 0.0000008158614264
Epoch: 19360/20000, Loss: 0.0000004974666012
Epoch: 19370/20000, Loss: 0.0000005542364079
Epoch: 19380/20000, Loss: 0.0000019207591322
Epoch: 19390/20000, Loss: 0.0000054788743000
Epoch: 19400/20000, Loss: 0.0000060953780121
Epoch: 19410/20000, Loss: 0.0000013582404108
Epoch: 19420/20000, Loss: 0.0000008318204436
Epoch: 19430/20000, Loss: 0.0000005845388387
Epoch: 19440/20000, Loss: 0.0000004944359944
Epoch: 19450/20000, Loss: 0.0000006782111086
Epoch: 19460/20000, Loss: 0.0000072902694228
Epoch: 19470/20000, Loss: 0.0000008594490737
Epoch: 194

In [9]:
print(test_tensor.shape)
prediction_tensor = torch.zeros(1, 20, 256).float()
print(prediction_tensor.shape)

torch.Size([1, 1, 256])
torch.Size([1, 20, 256])


In [10]:
with torch.no_grad():
    prediction = lem(test_tensor)
    prediction = prediction.view(1, 1, 256).float()
    prediction_tensor[:, 0, :] = prediction
    for i in range(19):
        prediction = lem(prediction)
        prediction = prediction.view(1, 1, 256).float()
        prediction_tensor[:, i+1, :] = prediction
        

### Four different types of error

In [11]:
# Exact Solution

u_test = u_1.T
u_test_full = u_test[80:100, :]

In [12]:
prediction_tensor = torch.squeeze(prediction_tensor)


In [13]:
# Extrapolation

k1 = ( prediction_tensor - u_test_full)**2
u_test_full_tensor = torch.tensor(u_test_full**2)
u_test_full_tensor.shape

torch.Size([20, 256])

### L^2 norm error

In [14]:
# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean(k1)/ torch.mean(u_test_full_tensor)

print("Relative Error Test: ", relative_error_test.item(), "%")

Relative Error Test:  0.010644946523613044 %


In [15]:
2+

SyntaxError: invalid syntax (4209523232.py, line 1)

### Max absolute norm error

In [ ]:
R_abs = torch.max(torch.abs(prediction_tensor - u_test_full))

In [ ]:
print(R_abs)

### Explained variance score

In [ ]:
import torch

a = prediction_tensor
b = u_test_full
# Assuming 'a' is your predicted values (model's predictions) and 'b' is the true values (ground truth)
# Make sure 'a' and 'b' are PyTorch tensors
# a = torch.tensor(a)
b = torch.tensor(b)
# Calculate the mean of 'b'
mean_b = torch.mean(b)

# Calculate the Explained Variance Score
numerator = torch.var(b - a)  # Variance of the differences between 'b' and 'a'
denominator = torch.var(b)    # Variance of 'b'
evs = 1 - numerator / denominator

print("Explained Variance Score:", evs.item())

### Mean absolute error

In [ ]:
# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean(torch.abs(prediction_tensor - u_test_full))

print("Relative Error Test: ", relative_error_test, "%")

### Contour plot for PINN (80 percent) and (20 percentage lem prediction)

In [ ]:
print(prediction_tensor.shape)
prediction_tensor = torch.squeeze(prediction_tensor)
input_tensor = torch.squeeze(input_tensor)

conc_u = torch.squeeze(input_tensor)
concatenated_tensor = torch.cat((conc_u, prediction_tensor), dim=0)

x1 = np.linspace(-1, 1, 256)
t1 = np.linspace(0, 1, 99)

### Snapshot time plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Create the figure and axis objects with reduced width
fig, ax = plt.subplots(figsize=(5, 5))  # You can adjust the width (7 inches) and height (5 inches) as needed




final_time_output = prediction_tensor[3, :]
final_out = final_time_output.detach().numpy().reshape(-1, 1)
final_true = u_1[:, 83].reshape(-1, 1)

# Plot the data with red and blue lines, one with dotted and one with solid style
ax.plot(x, final_out, color='red', linestyle='dotted', linewidth=12, label='Prediction')
ax.plot(x, final_true, color='blue', linestyle='solid', linewidth=7, label='True')


# Set the axis labels with bold font weight
ax.set_xlabel(r"${x}$", fontsize=26, color='black', fontdict={'weight': 'bold'})
ax.set_ylabel(r"${u(x, t)}$", fontsize=26, color='black', fontdict={'weight': 'bold'})

# Set the title with bold font weight
ax.set_title(r"${t = 0.83}$", fontsize=26, color='black', fontweight='bold')

# Set the number of ticks for x-axis and y-axis to 3
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Set tick labels fontweight to bold and increase font size
ax.tick_params(axis='both', which='major', labelsize=20, width=2, length=10)

# # Set the fontweight for tick labels to bold
# for tick in ax.get_xticklabels() + ax.get_yticklabels():
#     tick.set_weight('bold')

# Set the spines linewidth to bold
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)

# Set the legend
# ax.legend()

plt.savefig('LEM_0.83_20.pdf', dpi=500, bbox_inches="tight")
#plt.savefig('lem_0.83_20.png', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Create the figure and axis objects with reduced width
fig, ax = plt.subplots(figsize=(5, 5))  # You can adjust the width (7 inches) and height (5 inches) as needed



final_time_output = prediction_tensor[-2, :]
final_out = final_time_output.detach().numpy().reshape(-1, 1)
final_true = u_1[:, -2].reshape(-1, 1)

# Plot the data with red and blue lines, one with dotted and one with solid style
ax.plot(x, final_out, color='red', linestyle='dotted', linewidth=12, label='Prediction')
ax.plot(x, final_true, color='blue', linestyle='solid', linewidth=7, label='True')


# Set the axis labels with bold font weight
ax.set_xlabel(r"${x}$", fontsize=26, color='black', fontdict={'weight': 'bold'})
ax.set_ylabel(r"${u(x, t)}$", fontsize=26, color='black', fontdict={'weight': 'bold'})

# Set the title with bold font weight
ax.set_title(r"${t = 0.98}$", fontsize=26, color='black', fontweight='bold')

# Set the number of ticks for x-axis and y-axis to 3
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Set tick labels fontweight to bold and increase font size
ax.tick_params(axis='both', which='major', labelsize=20, width=2, length=10)

# # Set the fontweight for tick labels to bold
# for tick in ax.get_xticklabels() + ax.get_yticklabels():
#     tick.set_weight('bold')

# Set the spines linewidth to bold
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)

# Set the legend
# ax.legend()

plt.savefig('LEM_0.98_20.pdf', dpi=500, bbox_inches="tight")
#plt.savefig('lem_0.98_20.png', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


### Contour plot where 80 percent for PINN solution and 20 percent for lem solution

### Exact contour

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator

# Assuming you have defined concatenated_tensor as a PyTorch tensor
# concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)

# Convert concatenated_tensor to a NumPy array
concatenated_array = u_1.T

# Define custom color levels
x = np.linspace(-1, 1, concatenated_array.shape[1])  # Replace 0 and 1 with your actual x range
t = np.linspace(0, 1, concatenated_array.shape[0])  # Replace 0 and 1 with your actual t range
X, T = np.meshgrid(x, t)

# Define custom color levels using the minimum and maximum from the NumPy array
c_levels = np.linspace(np.min(concatenated_array), np.max(concatenated_array), 400)

# Plot the contour with interpolated data
plt.figure(figsize=(20, 5))
plt.pcolormesh(T, X, concatenated_array, shading='auto', cmap='coolwarm')

# Set the fontweight for axis labels to regular (not bold)
plt.xlabel("$t$", fontsize=26)
plt.ylabel("$x$", fontsize=26)
plt.title("$u(x, t)$", fontsize=26)

# Set tick labels fontweight to regular (not bold) and increase font size
plt.tick_params(axis='both', which='major', labelsize=20, width=3, length=10)

# Set the fontweight for tick labels to regular (not bold)
for tick in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
    tick.set_weight('normal')

# Set the number of ticks for x-axis and y-axis to 5
num_ticks = 5
x_ticks = np.linspace(np.min(T), np.max(T), num_ticks)
y_ticks = np.linspace(np.min(X), np.max(X), num_ticks)

plt.gca().xaxis.set_major_locator(FixedLocator(x_ticks))
plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))

cbar1 = plt.colorbar()
# Set the number of ticks for the color bar with uniformly distributed numbers
num_ticks = 5
c_ticks = np.linspace(np.min(concatenated_array), np.max(concatenated_array), num_ticks)
cbar1.set_ticks(c_ticks)

# Set the fontweight and fontsize for color bar tick labels
for t in cbar1.ax.get_yticklabels():
    t.set_weight('normal')
    t.set_fontsize(26)  # Increase the font size for color bar tick labels

# Increase the size of numbers on axis and color bar
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)

# Increase the tick size and width of the color bar
cbar1.ax.tick_params(axis='both', which='major', labelsize=30, width=3,  length=10)

#plt.savefig('Contour_Exact.pdf', dpi=500, bbox_inches="tight")
plt.savefig('contour_exact.jpeg', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator

# Assuming you have defined concatenated_tensor as a PyTorch tensor
# concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)

# Convert concatenated_tensor to a NumPy array
concatenated_array = concatenated_tensor.numpy()

# Define custom color levels
x = np.linspace(-1, 1, concatenated_array.shape[1])  # Replace 0 and 1 with your actual x range
t = np.linspace(0, 1, concatenated_array.shape[0])  # Replace 0 and 1 with your actual t range
X, T = np.meshgrid(x, t1)

# Define custom color levels using the minimum and maximum from the NumPy array
c_levels = np.linspace(np.min(concatenated_array), np.max(concatenated_array), 400)

# Plot the contour with interpolated data
plt.figure(figsize=(20, 5))
plt.pcolormesh(T, X, concatenated_array, shading='auto', cmap='coolwarm')

# Set the fontweight for axis labels to regular (not bold)
plt.xlabel("$t$", fontsize=26)
plt.ylabel("$x$", fontsize=26)
plt.title("$u(x, t)$", fontsize=26)

# Set tick labels fontweight to regular (not bold) and increase font size
plt.tick_params(axis='both', which='major', labelsize=20, width=3, length=10)

# Set the fontweight for tick labels to regular (not bold)
for tick in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
    tick.set_weight('normal')

# Set the number of ticks for x-axis and y-axis to 5
num_ticks = 5
x_ticks = np.linspace(np.min(T), np.max(T), num_ticks)
y_ticks = np.linspace(np.min(X), np.max(X), num_ticks)

plt.gca().xaxis.set_major_locator(FixedLocator(x_ticks))
plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))

cbar1 = plt.colorbar()
# Set the number of ticks for the color bar with uniformly distributed numbers
num_ticks = 5
c_ticks = np.linspace(np.min(concatenated_array), np.max(concatenated_array), num_ticks)
cbar1.set_ticks(c_ticks)

# Set the fontweight and fontsize for color bar tick labels
for t in cbar1.ax.get_yticklabels():
    t.set_weight('normal')
    t.set_fontsize(26)  # Increase the font size for color bar tick labels

# Increase the size of numbers on axis and color bar
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)

# Increase the tick size and width of the color bar
cbar1.ax.tick_params(axis='both', which='major', labelsize=30, width=3,  length=10)

# Add a dotted line at t = 0.8
plt.axvline(x=0.8, color='black', linestyle='dotted', linewidth=5)

#plt.savefig('Contour_LEM_20.pdf', dpi=500, bbox_inches="tight")
plt.savefig('contour_LEM_20.jpeg', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()
